In [3]:
from pathlib import Path

import os
import sys

os.environ['VECTORIAN_CPP_IMPORT'] = "1"
vectorian_path = Path("/Users/arbeit/Projects/vectorian-2021")
sys.path.append(str(vectorian_path))



from vectorian.importers import StringImporter
from vectorian.embeddings import SentenceBertEmbedding
from vectorian.corpus import Corpus
from tqdm import tqdm

import json
import spacy_sentence_bert

nlp = spacy_sentence_bert.load_model('en_paraphrase_distilroberta_base_v1')

with open("../data/raw_data/gold.json", "r") as f:
    queries = json.loads(f.read())

bert_embedding = SentenceBertEmbedding(nlp)

def prepare_docs():
    if bert_embedding is not None:
        im = StringImporter(nlp, embeddings=[bert_embedding])
    else:
        im = StringImporter(nlp)

    docs = []

    for query in tqdm(queries, desc="Importing"):
        for m in query["matches"]:
            docs.append(im(
                m["context"],
                title=m["work"],
                author=m["author"],
                unique_id=m["id"]))
            
    Corpus(docs).save("../data/processed_data/corpus")
    
prepare_docs()

Importing: 100%|██████████| 20/20 [03:25<00:00, 10.26s/it]
